In [2]:
!pip install memory_profiler

     |████████████████████████████████| 460 kB 1.8 MB/s eta 0:00:01
  Created wheel for memory-profiler: filename=memory_profiler-0.57.0-py3-none-any.whl size=28992 sha256=0a25369679a3664b367eb0489680d7c5dabaeef200d73658fa6b5379e0869dbe
  Stored in directory: /home/dzzp/.cache/pip/wheels/75/72/f3/ea9ac04a4cac6495412f33f260920a97e1eb185cbd3c839880
  Created wheel for psutil: filename=psutil-5.7.2-cp36-cp36m-linux_x86_64.whl size=279987 sha256=efd5901ec88b6981aecd014cb1e7064ee6b8c3fe5f391703a4bf37fd2646f187
  Stored in directory: /home/dzzp/.cache/pip/wheels/4c/1f/18/c066f071b2c2b638e40889c5595bc6b61fbe45eb475d29d61d
Successfully built memory-profiler psutil


In [4]:
!pip install ipython_memory_usage

  Created wheel for ipython-memory-usage: filename=ipython_memory_usage-1.2-py3-none-any.whl size=8397 sha256=d2f27271e96873b1ba821bc7819a553e88fccc310a15d2436bc659462069472e
  Stored in directory: /home/dzzp/.cache/pip/wheels/97/02/78/4c9d49eab56e23128172f74e0f853632048628a1ee2dabaebb
Successfully built ipython-memory-usage


In [1]:
# magic command %memit 를 사용하기 위해 필요한 라이브러리, 
import memory_profiler 
%load_ext memory_profiler
########################
# ipython_memory를 사용하여 cell별로 메모리를 계속 관찰하기 위한 라이브러리. 
import ipython_memory_usage 
import ipython_memory_usage.ipython_memory_usage as imu
%ipython_memory_usage_start

# memory watching start 
imu.start_watching_memory()

In [1] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 50.31 MiB


In [2]:
import os, sys

sys.path.append('/home/dzzp/workspace/minyoung/self-supervised-nas/libs/SemiNAS/nasbench')
sys.path.append('/home/dzzp/workspace/minyoung/self-supervised-nas/libs/NAS-Bench-201/nas_201_api')
sys.path.append('/home/dzzp/workspace/minyoung/self-supervised-nas/libs/nasbench')

In [2] used 0.0742 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 50.22 MiB


In [3]:
from model_spec_201 import *

In [3] used 22.6523 MiB RAM in 0.31s, peaked 0.00 MiB above current, total RAM usage 72.87 MiB


In [25]:
def convert_to_matrix_ops(arch_str):
    search_space = [ 'none', 'skip_connect', 'nor_conv_1x1', 'nor_conv_3x3', 'avg_pool_3x3']
    node_strs = arch_str.split('+')
    num_nodes = len(node_strs) + 1
    org_end_node = []
    matrix = np.zeros((num_nodes, num_nodes))

    converted_nodes = 1;
    to_node_dict = defaultdict(list)
    to_node_dict[0] = [0]
    converted_ops = ['input']

    for i, node_str in enumerate(node_strs):
        inputs = list(filter(lambda x: x != '', node_str.split('|')))
        for xinput in inputs: assert len(xinput.split('~')) == 2, 'invalid input length : {:}'.format(xinput)
        for xi in inputs:
            op, idx = xi.split('~')
            if op not in search_space: raise ValueError('this op ({:}) is not in {:}'.format(op, self.search_space))
            op_idx, node_idx = search_space.index(op), int(idx)
            matrix[i+1, node_idx] = op_idx
            if op_idx != 0:
                to_node_dict[i+1].append(converted_nodes)
                converted_ops.append(op)
                converted_nodes += 1
        if len(to_node_dict[i+1]) == 0 :
            to_node_dict[i+1].append(0)            
    
    for i in range(num_nodes):
        if np.count_nonzero(matrix[:,i]) == 0:
            org_end_node.append(i) 

    converted_ops.append('output')
    converted_nodes += 1      
    converted_end_node = converted_nodes - 1
    converted_matrix = np.zeros((converted_nodes, converted_nodes))
    for i in range(1, matrix.shape[0]):
        turn = 0
        for j in range(i):
            if matrix[i][j] != 0:
                to_node = to_node_dict[i][turn]
                for k in to_node_dict[j]:
                    converted_matrix[k][to_node] = 1
                    converted_matrix[to_node][k] = 1
                turn += 1
    print(org_end_node)
    for end_node in org_end_node:         
        for k in to_node_dict[end_node]:
            converted_matrix[k][converted_end_node] = 1
            converted_matrix[converted_end_node][k] = 1

    return converted_matrix, converted_ops

In [25] used 0.0117 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 74.69 MiB


In [26]:
model_str_list = ['|nor_conv_1x1~0|+|none~0|none~1|+|none~0|none~1|skip_connect~2|',
             '|nor_conv_3x3~0|+|nor_conv_1x1~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|',
             '|none~0|+|none~0|none~1|+|skip_connect~0|nor_conv_3x3~1|skip_connect~2|',
             '|none~0|+|avg_pool_3x3~0|none~1|+|avg_pool_3x3~0|skip_connect~1|nor_conv_1x1~2|',
             '|nor_conv_1x1~0|+|skip_connect~0|nor_conv_1x1~1|+|skip_connect~0|avg_pool_3x3~1|nor_conv_3x3~2|',
             '|nor_conv_1x1~0|+|nor_conv_3x3~0|nor_conv_3x3~1|+|avg_pool_3x3~0|avg_pool_3x3~1|none~2|',
             '|avg_pool_3x3~0|+|skip_connect~0|nor_conv_3x3~1|+|avg_pool_3x3~0|none~1|avg_pool_3x3~2|',
             '|none~0|+|none~0|avg_pool_3x3~1|+|nor_conv_1x1~0|nor_conv_1x1~1|none~2|',
             '|nor_conv_3x3~0|+|skip_connect~0|nor_conv_1x1~1|+|nor_conv_1x1~0|nor_conv_3x3~1|avg_pool_3x3~2|',
             '|nor_conv_1x1~0|+|skip_connect~0|skip_connect~1|+|nor_conv_3x3~0|skip_connect~1|none~2|',
             '|nor_conv_1x1~0|+|none~0|avg_pool_3x3~1|+|avg_pool_3x3~0|avg_pool_3x3~1|avg_pool_3x3~2|']

In [26] used 0.0000 MiB RAM in 0.10s, peaked 0.00 MiB above current, total RAM usage 74.69 MiB


In [27]:
for model_str in model_str_list:
    print(model_str)
    print()
    print(convert_to_matrix_ops(model_str))
    print("-"*100)

|nor_conv_1x1~0|+|none~0|none~1|+|none~0|none~1|skip_connect~2|

matrix
 [[0. 0. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]]
0  -  [0]
1  -  [1]
2  -  [0]
3  -  [2]
[1, 3]
(array([[0., 1., 1., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 1.],
       [0., 1., 1., 0.]]), ['input', 'nor_conv_1x1', 'skip_connect', 'output'])
----------------------------------------------------------------------------------------------------
|nor_conv_3x3~0|+|nor_conv_1x1~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|

matrix
 [[0. 0. 0. 0.]
 [3. 0. 0. 0.]
 [2. 0. 0. 0.]
 [3. 3. 2. 0.]]
0  -  [0]
1  -  [1]
2  -  [2]
3  -  [3, 4, 5]
[3]
(array([[0., 1., 1., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 1., 1., 1., 0.]]), ['input', 'nor_conv_3x3', 'nor_conv_1x1', 'nor_conv_3x3', 'nor_conv_3x3', 'nor_conv_1x1', 'o

In [ ]:
from nas_201_api import NASBench201API as API

dir_path = '/home/dzzp/workspace/dataset/'
nasbench_api = API(dir_path + 'NAS-Bench-201-v1_1-096897.pth')

try to create the NAS-Bench-201 api from /home/dzzp/workspace/dataset/NAS-Bench-201-v1_1-096897.pth


In [36]:
import random

arch_list = list(nasbench_api.archstr2index.keys())
tot = len(arch_list)
print(tot)

15625


In [45]:
for i in range(10):
    chosen_arch = random.choice(arch_list)
    print(chosen_arch)
    print()
    print(convert_to_matrix_ops(chosen_arch))
    print('\n')

|nor_conv_3x3~0|+|nor_conv_1x1~0|none~1|+|nor_conv_3x3~0|nor_conv_3x3~1|nor_conv_1x1~2|

(array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 1.],
       [0., 1., 1., 1., 1., 1., 0.]]), ['input', 'nor_conv_3x3', 'nor_conv_1x1', 'nor_conv_3x3', 'nor_conv_3x3', 'nor_conv_1x1', 'output'])


|none~0|+|none~0|none~1|+|skip_connect~0|nor_conv_3x3~1|skip_connect~2|

(array([[0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 1., 1., 0.]]), ['input', 'skip_connect', 'nor_conv_3x3', 'skip_connect', 'output'])


|none~0|+|avg_pool_3x3~0|none~1|+|avg_pool_3x3~0|skip_connect~1|nor_conv_1x1~2|

(array([[0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 1.],
       

In [46]:
%connect_info

{
  "shell_port": 48421,
  "iopub_port": 56673,
  "stdin_port": 56211,
  "control_port": 55131,
  "hb_port": 45113,
  "ip": "127.0.0.1",
  "key": "c9ca6ed8-e3bf96865856bd1c229167c7",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-0eb0f213-e779-4404-863c-c615bdba7251.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
